In [28]:
from typing import List


class Solution:
    # recursive solution
    def maxProfit(self, prices: List[int]) -> int:
        def helper(prices, i, has_stock) -> int:
            if i == len(prices):
                return 0
            # if we currently have the stock, we can either sell it for the current price or wait for the next day
            # if we don't have the stock, we can either buy it today or wait for the next day
            if has_stock:
                return max(helper(prices, i+1, False) + prices[i], helper(prices, i+1, True))
            else:
                return max(helper(prices, i+1, True) - prices[i], helper(prices, i+1, False))
            
        return helper(prices, 0, False)
        
    # memoization
    def maxProfit(self, prices: List[int]) -> int:
        memo = {}

        def helper(prices, i, has_stock) -> int:
            if i == len(prices):
                return 0
            # if we currently have the stock, we can either sell it for the current price or wait for the next day
            # if we don't have the stock, we can either buy it today or wait for the next day
            if has_stock:
                if not (i, has_stock) in memo:
                    memo[(i, has_stock)] = max(helper(prices, i+1, False) + prices[i], helper(prices, i+1, True))
                return memo[(i, has_stock)]
            else:
                if not (i, has_stock) in memo:
                    memo[(i, has_stock)] = max(helper(prices, i+1, True) - prices[i], helper(prices, i+1, False))
                return memo[(i, has_stock)]
            
        return helper(prices, 0, False)
    
    # dynamic programming, O(n) time, O(n) space
    def maxProfit(self, prices: List[int]) -> int:
        # most profitable way to have/not have stock at each day
        dp = [[0, 0] for _ in range(len(prices))]
        dp[0] = [-prices[0], 0]
        for i in range(1, len(prices)):
            # hold stock = previous hold or buy today
            dp[i][0] = max(dp[i-1][0], dp[i-1][1] - prices[i])
            # no stock = previous no stock or sell today
            dp[i][1] = max(dp[i-1][1], dp[i-1][0] + prices[i])

        # always best to have no stock at the end
        return dp[-1][1]

    # since we only ever use the previous day's values, we can reduce space complexity to O(1)
    # dynamic programming, O(n) time, O(1) space
    def maxProfit(self, prices: List[int]) -> int:
        dp = [-prices[0], 0]
        for i in range(1, len(prices)):
            # hold stock = previous hold or buy today
            dp[0] = max(dp[0], dp[1] - prices[i])
            # no stock = previous no stock or sell today
            dp[1] = max(dp[1], dp[0] + prices[i])
        
        # always best to have no stock at the end
        return dp[1]

In [29]:
import time
import random
import sys

random.seed(0)
sys.setrecursionlimit(10**6)

# test 1
prices = [7,1,5,3,6,4]
print("profit:", Solution().maxProfit(prices), "expected:", 7)

# test 2
prices = [1,2,3,4,5]
print("profit:", Solution().maxProfit(prices), "expected:", 4)

# test 3
prices = [7,6,4,3,1]
print("profit:", Solution().maxProfit(prices), "expected:", 0)

# test 4
n = 10
prices = [random.randint(1, 10) for _ in range(n)]
start = time.time()
print("profit:", Solution().maxProfit(prices))
print("time:", time.time()-start)

# test 5
n = 100
prices = [random.randint(1, 10) for _ in range(n)]
start = time.time()
print("profit:", Solution().maxProfit(prices))
print("time:", time.time()-start)

# test 6
n = 1000
prices = [random.randint(1, 10) for _ in range(n)]
start = time.time()
print("profit:", Solution().maxProfit(prices))
print("time:", time.time()-start)

# test 7
n = 10000
prices = [random.randint(1, 10) for _ in range(n)]
start = time.time()
print("profit:", Solution().maxProfit(prices))
print("time:", time.time()-start)

profit: 7 expected: 7
profit: 4 expected: 4
profit: 0 expected: 0
profit: 11
time: 5.817413330078125e-05
profit: 170
time: 0.0002880096435546875
profit: 1635
time: 0.0004208087921142578
profit: 16549
time: 0.0033159255981445312


In [32]:
from typing import List

# lets practice converting recursive solutions to dynamic programming
# recursive -> iterative -> dp -> optimized dp

class Solution:
    # recursive solution
    def maxProfitRecursive(self, prices: List[int]) -> int:
        def helper(prices, i, has_stock) -> int:
            if i == len(prices):
                return 0
            # if we currently have the stock, we can either sell it for the current price or wait for the next day
            # if we don't have the stock, we can either buy it today or wait for the next day
            if has_stock:
                return max(helper(prices, i+1, False) + prices[i], helper(prices, i+1, True))
            else:
                return max(helper(prices, i+1, True) - prices[i], helper(prices, i+1, False))
            
        return helper(prices, 0, False)
    
    # iterative solution
    def maxProfitIterative(self, prices: List[int]) -> int:
        stack = [(0, False, 0)] # (day, stock, profit)
        max_profit = 0
        while stack:
            day, stock, profit = stack.pop()
            if (day == len(prices)):
                max_profit = max(max_profit, profit)
                continue
            
            if stock:
                stack.append((day+1, False, profit+prices[day]))
                stack.append((day+1, True, profit))
            else:
                stack.append((day+1, True, profit-prices[day]))
                stack.append((day+1, False, profit))
        
        return max_profit
    
    # dynamic programming solution
    def maxProfitDP(self, prices: List[int]) -> int:
        # variables are day and stock indexing profit
        # add an extra day to make sure we sell
        dp = [[0, 0] for _ in range(len(prices)+1)]
        
        # before the first day, the most profitable way to have/not have stock is:
        # no stock = 0 profit
        dp[0][0] = 0
        # have stock = -prices[0] profit
        dp[0][1] = -prices[0]

        for i in range(1, len(prices)+1):
            dp[i][0] = max(dp[i-1][0], dp[i-1][1] + prices[i-1]) # no stock <- max(no stock, sell stock)
            dp[i][1] = max(dp[i-1][1], dp[i-1][0] - prices[i-1]) # have stock <- max(have stock, buy stock)
        return dp[-1][0]
    
    # optimized dynamic programming solution
    def maxProfit(self, prices: List[int]) -> int:
        # variables are day and stock indexing profit
        # in the previous solution, we only ever use the previous day's values
        # so we can reduce space complexity to O(1)
        dp = [0, -prices[0]]
        for i in range(1, len(prices)+1):
            dp[0] = max(dp[0], dp[1] + prices[i-1])
            dp[1] = max(dp[1], dp[0] - prices[i-1])
        return dp[0]
            

In [33]:
import time
import random
import sys

random.seed(0)
sys.setrecursionlimit(10**6)

solution = Solution()
f = solution.maxProfit

# test 1
prices = [7,1,5,3,6,4]
print("profit:", f(prices), "expected:", 7)

# test 2
prices = [1,2,3,4,5]
print("profit:", f(prices), "expected:", 4)

# test 3
prices = [7,6,4,3,1]
print("profit:", f(prices), "expected:", 0)

# test 4
n = 10
prices = [random.randint(1, 10) for _ in range(n)]
start = time.time()
print("profit:", f(prices))
print("time:", time.time()-start)

# test 5
n = 100
prices = [random.randint(1, 10) for _ in range(n)]
start = time.time()
print("profit:", f(prices))
print("time:", time.time()-start)

# test 6
n = 1000
prices = [random.randint(1, 10) for _ in range(n)]
start = time.time()
print("profit:", f(prices))
print("time:", time.time()-start)

# test 7
n = 10000
prices = [random.randint(1, 10) for _ in range(n)]
start = time.time()
print("profit:", f(prices))
print("time:", time.time()-start)

profit: 7 expected: 7
profit: 4 expected: 4
profit: 0 expected: 0
profit: 11
time: 5.6743621826171875e-05
profit: 170
time: 8.225440979003906e-05
profit: 1635
time: 0.0004570484161376953
profit: 16549
time: 0.00384521484375
